<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/vae_faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install vae_utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement vae_utils (from versions: none)
ERROR: No matching distribution found for vae_utils


In [14]:
import os
from zipfile import ZipFile

from google.colab import files

# Upload the zip file to Google Colab
uploaded = files.upload()



Saving vae_utils.py to vae_utils (1).py


In [16]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import (
    layers,
    models,
    callbacks,
    utils,
    metrics,
    losses,
    optimizers,
)

from scipy.stats import norm
import pandas as pd

from vae_utils import get_vector_from_label , add_vector_to_images, morph_faces


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


0.Parameters

In [17]:
IMAGE_SIZE = 64
CHANNELS = 3
BATCH_SIZE = 128
NUM_FEATURES = 64
Z_DIM = 200
LEARNING_RATE = 0.0005
EPOCHS = 10
BETA = 2000
LOAD_MODEL = False

1.Prepare The Data

In [19]:
!pip install kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.13-py3-none-any.whl size=77717 sha256=115c9457addc112abff5ffa64dbc4c0eca8abf74ed4fabc3b1eaec60e86696e6
  Stored in directory: /root/.cache/pip/wheels/f3/16/ff/34e7d368370d4fd68bb749a59f1d2639ed66f3c14358e340a1
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.13
    Uninstalling kaggle-1.5.13:
      Successfully uninstalled kaggle-1.5.13


In [20]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

!kaggle datasets download -d {https://www.kaggle.com/datasets/jessicali9530/celeba-dataset}

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /content. Or use the environment method.


In [1]:
#Load the data
train_data = utils.image_dataset_from_directory(
    "/app/data/celeba-datasset/img_align_celeba/img_align_celeba",

    labels = None,
    color_mode = "rgb",
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE ,
    shuffle = True,
    seed = 42 ,
    interpolation = "bilinear",
                  )


NameError: ignored

In [2]:
#Preprocess the data
def preprocess(img):
  img = tf.cast(img , "float32") / 255.0
  return img

train = train_data.map(lambda x: preprocess(x))

NameError: ignored

In [3]:
train_sample = sample_batch(train)


NameError: ignored

In [4]:
#Show some features from the training set
display(train_sample , cmap = None)

NameError: ignored

## 2.Build the variational Autoencoder

In [5]:
class Sampling(layers.layer):
  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = K.random_normal(shape = (batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

NameError: ignored

In [2]:
#Encoder
encoder_input = layers.Input(
    shape = (IMAGE_SIZE , IMAGE_SIZE , CHANNELS), name = "encoder_input"
)
x = layers.Conv2D(NUM_FEATURES, kernel_size = 3, strides = 2, padding= "same")(
    encoder_input
)

x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(NUM_FEATURES, kernel_size = 3, strides = 2, padding = "same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(NUM_FEATURES, kernel_size = 3, strides = 2, padding = "same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(NUM_FEATURES, kernel_size = 3, strides = 2, padding = "same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(NUM_FEATURES, kernel_size = 3, strides = 2 , padding = "same")
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
shape_before_flattening  = K.int_shape(x)[1:]   #The decoder will need this

x = layers.Flatten()(x)
z_mean = layers.Dense(Z_DIM, name= "z_mean")(x)
z_log_var = layers.Dense(Z_DIM, name = "Z_LOG_VAR")(x)
z =Sampling()([z_mean, z_log_var])


encoder = models.Model(encoder_input, [z_mean, z_log_var , z], name = "encoder")
encoder.summary()

NameError: ignored

In [3]:
#Decoder
decoder_input = layers.Input(shape =( Z_DIM, ), name = "decoder_input")
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose( NUM_FEATURES, kernel_size = 3 , strides =2 , padding = "same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(NUM_FEATURES, kernel_size = 3, strides =2 , padding = "same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(NUM_FEATURES, kernel_size = 3 , strides = 2 , padding = "same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2dTranspose(NUM_FEATURES , kernel_size = 3 , strides = 2 , padding = "same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(NUM_FEATURES, kernel_size = 3, strides = 2 , padding = "same")( x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
decoder_output = layers.Conv2DTranspose(CHANNELS, kernel_size = 3 , strides = 1 , activation = "sigmoid" , padding = "same")(x)
decoder = models.Model(decoder_input, decoder_output)
decoder.summary()


NameError: ignored

In [4]:
class VAE(models.Model):
  def __init__ (self, encoder, decoder, **kwargs):
    super(VAE, self).__init__(**kwargs)
    self.encoder = encoder
    self.decoder = decoder
    self.total_loss_tracker = metrices.Mean(name = "total_loss")
    self.reconstruction_loss_tracker = metrices.Mean(
        name = "reconstruction_loss"
    )
    self.kl_loss_tracker = metrices.Mean(name = "kl_loss")

  @property
  def metrices(self):
    return[
        self.total_loss_tracker,
        self.reconstruction_loss_tracker,
        self.kl_loss_tracker,
    ]

  def call(self, inputs):
    """Call the model on a particular input"""
    z_mean , z_log_var, z = encoder(inputs)
    reconstruction = decoder(z)
    return z_mean , z_log_var, reconstruction

  def train_step(self, data):
    """Step run during training."""
    with tf.GradientTape() as tape :
      z_mean , z_log_var, reconstruction = self(data, training =True)
      reconstruction_loss = tf.reduce_mean(
          BETA * losses.mean_squared_error(data, reconstruction)
      )
      kl_loss = tf.reduce_mean(
          tf.reduce_Sum(
              -0.5
              * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
              axis = 1,
          )
      )
      total_loss = reconstruction_loss + kl_loss

    grades = tape.gradient(total_loss, self.trainable_weights)
    self.optimizer.apply_gradient(zip(grades, self.trainable_weights))

    self.total_loss_tracker.update_state(total_loss)
    self.reconstruction_loss_tracker.update_state(reconstruction_loss)
    self.kl_loss_tracker.update_state(kl_loss)

    return {
        "loss" :self.total_loss_tracker.result(),
        "reconstruction_loss" : self.reconstruction_loss_tracker.result(),
        "kl_loss" :self.kl_loss_tracker.result(),

    }

  def test_step(self, data):
    """Step run during validation."""

    if isinstance(data, tuple):
      data = data[0]

      z_mean, z_log_var , reconstruction = self(data)
      reconstruction_loss = tf.reduce_mean(
          BETA * losses.mean_squared_error(data, reconstruction)
      )
      kl_loss = tf.reduce_mean (
          tf.reduce_sum(
              -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
              axis = 1,
          )
      )
    total_loss = reconstruction_loss + kl_loss

    return {
        "loss": total_loss,
        "reconstruction_loss" : reconstruction_loss,
        "kl_loss" :kl_loss,
    }



NameError: ignored

In [5]:
#Create a variational autoencoder
vae = VAE(encoder , decoder)

NameError: ignored

# 3. Train the variational autoencoder

In [6]:
#compile the variational autoencoder
optimizer = optimizers.Adam(learning_rate = LEARNING_RATE)
vae.compile(optimizer = optimizer)


NameError: ignored

In [ ]:
#Create a model save checkpoint
model_checkpoint_callback = callbacks.model_checkpoint_callback(
    filepath = "./checkpoint",
    save_weights_only = False,
    save_freq = "epochs",
    monitor = "loss",
    mode ="min",
    save_best_only = True,
    verbose = 0,
)

tensorboard_callback = callbacks.TenosrBoard(log_dir = "./logs")


class ImageGenrator(callbacks.Callback):
  def __init__(self, num_img, latent_dim):
    self.num_img = num_img
    self.latent_dim = latent_dim